In [ ]:
import torch
from torch import einsum

'''cuda'''
print(torch.cuda.is_available())
print(torch.cuda.device_count())

'''triu tril'''
up = torch.triu(torch.ones(4, 4))
low = torch.tril(torch.ones(4, 4))
print(up)
print(low)

In [ ]:
'''view reshape '''
x = torch.ones(2, 3)
print(x.data_ptr() == x.reshape(1, 6).data_ptr())   # True
print(x.data_ptr() == x.transpose(0, 1).reshape(1, 6).data_ptr())   # False

print(x.view(1, 6).is_contiguous())         # True
print(x.reshape(1, 6).is_contiguous())      # True
print(x.flatten().is_contiguous())          # True
print(x.transpose(1, 0).is_contiguous())    # False
print(x.permute(1, 0).is_contiguous())      # False


'''flatten'''
x = torch.randn(4, 3, 10, 10)
y = torch.flatten(x, start_dim=0, end_dim=1)
print(y.shape)

'''chunk'''
x = torch.rand(8, 3, 2)
x = x.chunk(4, dim=0)           # 4 * [2, 3, 2]
x = [temp[None] for temp in x]  # 4 * [1, 2, 3, 2]
x = torch.cat(x, dim=0)
print(x.shape)  

In [ ]:
'''cat stack hstack vstack'''
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5, 6])
print(torch.cat((a, b), 0))       
print(torch.hstack((a, b)))
print(torch.vstack((a, b)))
print(torch.stack((a, b), 0))
print(torch.stack((a, b), 1))

a = torch.tensor([[1],[2],[3]])
b = torch.tensor([[4],[5],[6]])
print(torch.hstack((a, b)))
print(torch.vstack((a, b)))

x = torch.ones(2, 3)
y = torch.ones(2, 3) * 2
print(torch.cat((x, y), 0))
print(torch.cat((x, y), 1))
print(torch.stack((x, y), 0))
print(torch.stack((x, y), 1))
print(torch.hstack((x, y)))
print(torch.vstack((x, y)))

In [16]:
'''expand repeat'''
x = torch.tensor([1, 2, 3])

y = x.expand(2, 3)
print(y)
print(x.data_ptr() == y.data_ptr())
y[1, 2] = 0
print(x)

z = x.repeat(2, 3)
print(z)
print(x.data_ptr() == z.data_ptr())



tensor([[1, 2, 3],
        [1, 2, 3]])
True
tensor([1, 2, 0])
tensor([[1, 2, 0, 1, 2, 0, 1, 2, 0],
        [1, 2, 0, 1, 2, 0, 1, 2, 0]])
False


In [ ]:
'''transpose permute'''
x = torch.rand(2, 3, 4)

y = x.transpose(1, 2)
print(y.shape)

z = x.permute(2, 1, 0)
print(z.shape)


In [ ]:

'''squeeze unsqueeze'''
x = torch.tensor([[1], [2], [3], [4]])

y = x.squeeze()
print(y)

z = y.unsqueeze(0)
print(z)


In [ ]:
'''max argmax'''
# x = torch.rand(4, 4)
# print(x)
# print(torch.max(x, 0))
# print(torch.max(x, 1))

'''sort'''
x = torch.tensor([14, 16, 15, 17, 18])
y = torch.sort(x)
print(y)

'''topk'''
# x = torch.rand(4, 4)
# print(x)
# print(x.topk(1))

'''nonzero'''
# x = torch.tensor([
#     [1, 1, 0],
#     [0, 1, 1]
# ])
# print(torch.nonzero(x, as_tuple=True))

'''mean'''
# x = torch.ones(3, 4)
# print(x.mean(dim=0))
# print(x.mean(dim=1))


'''mul'''
# x = torch.ones(3, 4)
# y = torch.rand(3, 1)
# print(x * y) # torch.mul(x, y)

# x = torch.tensor([[1, 2], [3, 4]])
# y = torch.tensor([[1, 2], [3, 4]])
# print(x * y) # torch.mul(x, y)

'''einsum'''
# x = torch.ones(3, 2)
# y = torch.ones(4, 2) * 2
# z = einsum('ik, jk -> ij', x, y)
# print(z)

In [ ]:
import torch
import torch.nn as nn

'''softmax'''
softmax = nn.Softmax(dim=1)
x = torch.randn(3, 3)
y = softmax(x)
print(y)


'''AdpativeMaxPool'''
pool = nn.AdaptiveMaxPool2d(1)
x = torch.rand(1, 10, 9, 9)
y = pool(x)
print(y.shape)

pool = nn.AdaptiveMaxPool3d(1)
x = torch.rand(1, 10, 9, 9, 9)
y = pool(x)
print(y.shape)

'''AdpativeAvgPool'''
pool = nn.AdaptiveAvgPool2d(1)
x = torch.rand(1, 10, 9, 9)
y = pool(x)
print(y.shape)

pool = nn.AdaptiveAvgPool3d(1)
x = torch.rand(1, 10, 9, 9, 9)
y = pool(x)
print(y.shape)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pack_sequence, pad_packed_sequence

class MyDataSet(Dataset):
    def __init__(self):
        a = torch.tensor([1,2,3,4])
        b = torch.tensor([5,6,7])
        c = torch.tensor([7,8])
        d = torch.tensor([9])
        self.data = [a, b, c, d]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        print(idx)
        return self.data[idx]

def collate_fn(batch):
    print(batch)
    batch.sort(key=lambda x: len(x), reverse=True)
    seq_len = [s.size(0) for s in batch]
    batch = pad_sequence(batch, batch_first=True)
    return batch, seq_len

embed = nn.Embedding(10, 3, padding_idx=0)
dataset = MyDataSet()
dataloader = DataLoader(dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

data, seq_len = next(iter(dataloader))
print(data)

data = embed(data)
print(data)

data = pack_padded_sequence(data, seq_len, batch_first=True)
print(data)


In [ ]:
from torchvision.io import read_image
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torchvision import datasets, models, transforms

from torchvision.models.resnet import resnet18
from torchvision.models.video.resnet import r3d_18

resnet = resnet18()
resnet_3d = r3d_18()

print(resnet)